In [1]:
import warnings

import geopandas
import libpysal
import momepy
import osmnx
import pandas as pd
import configparser
import os

from clustergram import Clustergram

import matplotlib.pyplot as plt
from bokeh.io import output_notebook
from bokeh.plotting import show

output_notebook()

/opt/anaconda3/envs/momepy_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading BokehJS ...

In [ ]:
# Define in config file the file_path to open the dataframe
config = configparser.ConfigParser()
config_file = 'config.ini'
# check if file exists
if not os.path.isfile(config_file):
    # create a simple config file
    config['Paths'] = {'root_folder': ''}
    with open(config_file, 'w') as configfile:
        config.write(configfile)
    print(f"Created configuration file '{config_file}'.")

config.read('config.ini')

df_file_path = config['Paths']['df_file_path']  
buildings_file_path = config['Paths']['buildings_file_path']  
print(df_file_path)
print(buildings_file_path)

In [ ]:
df = pd.read_csv(df_file_path)
buildings = pd.read_csv(buildings_file_path)

In [ ]:
standardized = (df - df.mean()) / df.std()
standardized # print to see some values

In [ ]:
cgram = Clustergram(range(1, 14), n_init=10, random_state=0)
cgram.fit(standardized.fillna(0))

In [ ]:
show(cgram.bokeh())

In [ ]:
cgram.labels.head()

In [ ]:
df["cluster"] = cgram.labels[13].values

In [ ]:
urban_types = buildings[["geometry", "uID"]].merge(df[["uID", "cluster"]], on="uID")

In [ ]:
urban_types.explore("cluster", categorical=True, prefer_canvas=True, tiles="CartoDB Positron", tooltip=False)

In [ ]:
import matplotlib.patches as mpatches

# Define the colors for each cluster
colors = plt.get_cmap('tab20').colors
categories = urban_types['cluster'].unique()

# Create the legend handles
legend_handles = [mpatches.Patch(color=colors[i], label=f'Cluster {category}') 
                  for i, category in enumerate(categories)]

# Create separate plots for each attribute
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

# Plot for 'cluster' attribute
urban_types.plot(column='cluster', cmap='Set1', legend=False, ax=ax)

# Add title and customize plot
ax.set_title('Urban Types by Cluster', fontsize=16)
ax.set_axis_off()  # Optionally remove axis lines for a cleaner look

# Add the custom legend
ax.legend(handles=legend_handles, title='Cluster', bbox_to_anchor=(1, 1), loc='upper left')

# Display the plot
plt.show()